In [201]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false; // disable scroll bar when displaying Folium map
}

<IPython.core.display.Javascript object>

# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Global Historical Climatology Network daily (GHCNd)](https://www.ncei.noaa.gov/products/land-based-station/global-historical-climatology-network-daily) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe - it's a wonderfully large dataset to play with! In particular, you will be asked to use data from the Ann Arbor Michigan location (my home!). and this is stored in the file: `assets/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv`

Each row in this datafile corresponds to a single observation from a weather station, and has the following variables:
* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write a python notebook which plots line graphs of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015. (Based on the graph, do you think extreme weather is getting more frequent in 2015?)
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

I've written some steps I think would be good to go through, but there are other ways to solve this assignment so feel free to explore the pandas library! What I really want to see is an image that looks like this sketch I drew at my desk:

![](assets/chris_sketch.png)

In [2]:
#  I'll be using the folium package to render the data into a map in Jupyter.

import folium
import pandas as pd

# get the location information for this dataset
df = pd.read_csv('assets/BinSize_d400.csv')
station_locations_by_hash = df[df['hash'] == 'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89']

# get longitude and lattitude to plot
lons = station_locations_by_hash['LONGITUDE'].tolist()
lats = station_locations_by_hash['LATITUDE'].tolist()

# plot on a beautiful folium map
my_map = folium.Map(location = [lats[0], lons[0]], height = 500,  zoom_start = 9)
for lat, lon in zip(lats, lons):
    folium.Marker([lat, lon]).add_to(my_map)

# render map in Jupyter
display(my_map)

## Step 1
Load the dataset and transform the data into Celsius (refer to documentation) then extract all of the rows which have minimum or maximum temperatures.

__hint: when I did this step I had two DataFrame objects, each with ~80,000 entries in it__

In [143]:
import pandas as pd
df = pd.read_csv('assets/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv')
df.head()

,ID,Date,Element,Data_Value
0,USW00094889,2014-11-12,TMAX,22
1,USC00208972,2009-04-29,TMIN,56
2,USC00200032,2008-05-26,TMAX,278
3,USC00205563,2005-11-11,TMAX,139
4,USC00200230,2014-02-27,TMAX,-106


In [144]:
# In this code cell, transform the Data_Value column
# Transform temp data into Celsius
df['Data_Value'] = df['Data_Value'].astype(int).apply(lambda x:x/10)

# Separate the date to three columns for future date filter
df['year']  = pd.to_datetime(df['Date']).apply(lambda x:x.year).astype(int)
df['month'] = pd.to_datetime(df['Date']).apply(lambda x:x.month).astype(int)
df['day']   = pd.to_datetime(df['Date']).apply(lambda x:x.day).astype(int)

# Separate the df into 2 data frames by the 'Element' categories. 
df_max = df[df['Element']=='TMAX']
df_min = df[df['Element']=='TMIN']


## Step 2
In order to visualize the data we would plot the min and max data for each day of the year between the years 2005 and 2014 across all weather stations. But we also need to find out when the min or max temperature in 2015 falls below the min or rises above the max for the previous decade.

If you did step 1 you have two Series objects with min and max times for the years 2005 through 2015. You can use Pandas `groupby` to create max and min temperature Series objects across all weather stations for each day of these years, and you can deal with the records for February 29 (the leap year) by dropping them.

__hint: when I finished this step, I had two DataFrame objects, each with exactly 4015 observations in them__

In [153]:
import numpy as np

# Select the data from the year of 2005 to 2014
df_max_05_15 = df_max[(df_max['year']>=2005)&(df['year']<=2015)]
df_min_05_15 = df_min[(df_min['year']>=2005)&(df['year']<=2015)]

# Remove all the records of 29th Feb.
df_max_05_15 = df_max_05_15[~((df_max_05_15['month']==2)&(df_max_05_15['day']==29))]
df_min_05_15 = df_min_05_15[~((df_min_05_15['month']==2)&(df_min_05_15['day']==29))]

# create a DataFrame of maximum temperature by date
df_max_05_15 = df_max_05_15.groupby(['year','month','day']).max()
# create a DataFrame of minimum temperatures by date
df_min_05_15 = df_min_05_15.groupby(['year','month','day']).min()


ID        Date Element  Data_Value
year month day                                             
2005 1     1    USW00094889  2005-01-01    TMAX        15.6
           2    USW00094889  2005-01-02    TMAX        13.9
           3    USW00094889  2005-01-03    TMAX        13.3
           4    USW00094889  2005-01-04    TMAX         3.9
           5    USW00094889  2005-01-05    TMAX         3.3
...                     ...         ...     ...         ...
2015 12    27   USW00094889  2015-12-27    TMAX         8.3
           28   USW00094889  2015-12-28    TMAX         6.1
           29   USW00094889  2015-12-29    TMAX        10.0
           30   USW00094889  2015-12-30    TMAX         6.7
           31   USW00094889  2015-12-31    TMAX         1.7

[4015 rows x 4 columns]

## Step 3
Now that you have grouped the daily max and min temperatures for each day of the years 2005 through 2015, you can separate out the data for 2015. Then you can use the Pandas `groupby` function to find the max and min of the temperature data for each __day of the year__ for the 2005-2014 data.

__hint: at the end of this step I had two DataFrames, one of maximum and the other of minimum values, which each had 365 observations in them. I also had another pair of similar DataFrames but only for the year 2015.__

In [163]:
# calculate the minimum and maximum values for the day of the year for 2005 through 2014
max_05_14 = df_max_05_15[(df_max_05_15.index.get_level_values('year') >= 2005) & (df_max_05_15.index.get_level_values('year') < 2015)].groupby(['month','day']).aggregate({'Data_Value':np.max})
min_05_14 = df_min_05_15[(df_min_05_15.index.get_level_values('year') >= 2005)& (df_min_05_15.index.get_level_values('year') < 2015)].groupby(['month','day']).aggregate({'Data_Value':np.min})

# calculate the minimum and maximum values for the years 2015
max_15 = df_max_05_15[df_max_05_15.index.get_level_values('year') == 2015].groupby(['month','day']).aggregate({'Data_Value':np.max})
min_15 = df_min_05_15[df_max_05_15.index.get_level_values('year') == 2015].groupby(['month','day']).aggregate({'Data_Value':np.min})

# Find the data in 2015 which exceeded the previous 10 years.
exceeded_max = np.where(max_15['Data_Value'] > max_05_14['Data_Value'])
exceeded_min = np.where(min_15['Data_Value'] < min_05_14['Data_Value'])


(array([  4,  10,  33,  44,  45,  46,  47,  49,  50,  51,  53,  54,  55,
         56,  57,  58,  63,  64,  65,  86,  87,  88, 113, 114, 139, 183,
        239, 289, 290, 291, 292, 313]),)

## Step 4
Now it's time to plot! You need to explore matplotlib in order to plot line graphs of the min and max temperatures for the years 2005 through 2014 and to scatter plot __only__ the daily 2015 temperatures that exceeded those values.

In [200]:
import matplotlib.pyplot as plt
from calendar import month_abbr

# plotting code
plt.figure(figsize=(12, 10))

# the max and min temperature for each day derived from the 2005-2014.
plt.plot(max_05_14.values, label='Max Temp (2005-2014)', linewidth=1,alpha = 0.7,c='salmon')
plt.plot(min_05_14.values, label='Min Temp (2005-2014)', linewidth=1,alpha = 0.7,c='royalblue')

plt.gca().fill_between(range(len(max_05_14)), min_05_14['Data_Value'],max_05_14['Data_Value'], facecolor='lightslategrey', alpha=0.2)

# these exceeded temperature records from 2015.
plt.scatter(exceeded_max, max_15.iloc[exceeded_max], s=12, color='red', label='High Temp Exceeded (2015)')
plt.scatter(exceeded_min, min_15.iloc[exceeded_min], s=12, color='blue', label='Low Temp Exceeded (2015)')

# the legend 
plt.legend(loc = 'lower center', bbox_to_anchor=(0.5, 0.05), title='Temperature', fontsize=10)

# the tick names
plt.xticks(np.linspace(0,30 + 30*11 , num = 12), (r'Jan', r'Feb', r'Mar', r'Apr', r'May', r'Jun', r'Jul', r'Aug', r'Sep', r'Oct', r'Nov', r'Dec'), alpha = 0.8 )
plt.yticks(alpha = 0.8 )
plt.xlim(0,365)
plt.xlabel('Months', alpha = 0.8)
plt.ylabel('Temperature ($^\circ$C)', alpha = 0.8)
plt.title('The lowest & highest Temperature by Day of the Year', fontweight='bold', fontsize=14, alpha = 0.8)

# improve the data-ink ratio
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['bottom'].set_alpha(0.3)
plt.gca().spines['left'].set_alpha(0.3)
